In [2]:
import torch

In [ ]:
import torch_directml

In [3]:
import pandas as pd
from datasets import load_dataset
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from pathlib import Path

/home/happygaming/miniconda3/envs/dl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-03 07:40:07.656640: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-03 07:40:09.528742: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdirectml.d6f03b303ac3c4f2eeb8ca631688c9757b361310.so
2024-08-03 07:40:09.528831: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdxcore.so
2024-08-03 07:40:09.534320: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libd3d12.so
2024-08-03 07:40:09.7351

In [4]:
# Activate DirectML for Windows Subsystem Linux
dml = torch_directml.device()
tensor1 = torch.tensor([1]).to(dml) # Note that dml is a variable, not a string!
tensor2 = torch.tensor([2]).to(dml)
dml_algebra = tensor1 + tensor2
dml_algebra.item()

3

In [4]:
# Load dataset
dataset = load_dataset('csv', data_files='domain_vocab/small_vocab.txt')

Using custom data configuration default-b83ad8a621003a60


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

c:\Users\iohkg\miniconda3\envs\dl\lib\site-packages\datasets\packaged_modules\csv\csv.py:156: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


Dataset csv downloaded and prepared to C:\Users\iohkg\.cache\huggingface\datasets\csv\default-b83ad8a621003a60\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['domain', 'general'],
        num_rows: 324
    })
})

In [3]:
# Load tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
# Load model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

In [11]:
def preprocess_data(data):
    inputs = data['general']
    targets = data['domain']
    model_inputs = tokenizer(inputs, max_length=10, truncation=True, padding='max_length')
    
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=4, truncation=True,  padding='max_length')
    
    labels["input_ids"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in labels_example] 
        for labels_example in labels["input_ids"]
    ]

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [14]:
tokenized_dataset = dataset.map(preprocess_data, batched=True,  remove_columns=["general", "domain"])

Parameter 'function'=<function preprocess_data at 0x00000204CF71C310> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

c:\Users\iohkg\miniconda3\envs\dl\lib\site-packages\transformers\tokenization_utils_base.py:4144: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [15]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 324
    })
})

In [16]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=0.001,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['train'],
)

c:\Users\iohkg\miniconda3\envs\dl\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
# Fine-tune the model with large dataset
trainer.train()

  0%|          | 0/486 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [15]:
# Fine-tune the model with small dataset
trainer.train()

***** Running training *****
  Num examples = 324
  Num Epochs = 3
  Instantaneous batch size per device = 2


  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 486
  Number of trainable parameters = 406291456


Epoch,Training Loss,Validation Loss
1,No log,1.104275
2,No log,1.016484
3,No log,0.940805


***** Running Evaluation *****
  Num examples = 324
  Batch size = 2
***** Running Evaluation *****
  Num examples = 324
  Batch size = 2
***** Running Evaluation *****
  Num examples = 324
  Batch size = 2


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=486, training_loss=0.8041648707762667, metrics={'train_runtime': 1165.2983, 'train_samples_per_second': 0.834, 'train_steps_per_second': 0.417, 'total_flos': 24168295464960.0, 'train_loss': 0.8041648707762667, 'epoch': 3.0})

In [16]:
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

***** Running Evaluation *****
  Num examples = 324
  Batch size = 2


{'eval_loss': 0.9408054947853088, 'eval_runtime': 33.8424, 'eval_samples_per_second': 9.574, 'eval_steps_per_second': 4.787, 'epoch': 3.0}


In [17]:
# Save the fine-tuned model
model.save_pretrained('./fine-tuned-bart')
tokenizer.save_pretrained('./fine-tuned-bart')

Configuration saved in ./fine-tuned-bart/config.json


Model weights saved in ./fine-tuned-bart/pytorch_model.bin
tokenizer config file saved in ./fine-tuned-bart/tokenizer_config.json
Special tokens file saved in ./fine-tuned-bart/special_tokens_map.json


('./fine-tuned-bart/tokenizer_config.json',
 './fine-tuned-bart/special_tokens_map.json',
 './fine-tuned-bart/vocab.json',
 './fine-tuned-bart/merges.txt',
 './fine-tuned-bart/added_tokens.json')

In [5]:
import os
print(os.getcwd())

d:\Pop_Documents\workspace\github\pMage\src\main\python\case_study\brake_certification


In [5]:
model_bin = Path("D:/Pop_Documents/workspace/github/pMage_AI_model/fine-tuned-bart/pytorch_model.bin")
state_dict = torch.load(model_bin, map_location="cpu")

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [3]:
# Load the fine-tuned model and tokenizer
ft_model = BartForConditionalGeneration.from_pretrained('../../../../../../pMage_AI_model/fine-tuned-bart')
tokenizer = BartTokenizer.from_pretrained('../../../../../../pMage_AI_model/fine-tuned-bart')

OSError: Unable to load weights from pytorch checkpoint file for 'D:\Pop_Documents\workspace\github\pMage_AI_model\fine-tuned-bart\pytorch_model.bin' at 'D:\Pop_Documents\workspace\github\pMage_AI_model\fine-tuned-bart\pytorch_model.bin'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.

### Test generating summary of the task name

In [6]:
def summarize_phrase(ft_model, phrase, min_len=4, max_len=10):
    # Tokenize the input phrase
    inputs = tokenizer(phrase, max_length=150, truncation=True, return_tensors="pt")

    summary_ids = ft_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_len,
        min_length=min_len,
        num_beams=4,
        length_penalty=2.0,
        early_stopping=True
    )

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [18]:
input_phrase = "Assemble the system into a prototype vehicle that we can verify if the brake system can work properly via some specific assessments carried out by another team"

In [19]:
summary = summarize_phrase(ft_model, input_phrase, 10, 10)
print(summary)

test the system.Assemble


In [7]:
tasks = ["Design brake system blueprint", 
         "Implement Remote Sensing Devices braking activation", 
         "Connect Remote Sensing Devices with brake control system", 
         "Assemble the system into a prototype vehicle", 
         "Assemble the system into the final version"]

In [8]:
for task in tasks:
    summary = summarize_phrase(ft_model, task, 10, 10)
    summary = summary.replace(" ", "_")
    print(summary)

brakeke_system_design
brakebrake_activation
brakebrake_Vehicle
autom/brake
technician.Assemble
